In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [3]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [4]:
#dataset = os.path.join(parentdir,'ml-latest-small/ratings.1.csv')    
#ratings = pd.read_csv(dataset, sep=',',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)


dataset = os.path.join(parentdir,'movielens100k/u.data')     
ratings = pd.read_csv(dataset, sep='\s+',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/movielens100k/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

In [5]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
ratings1 = pd.merge(ratings, items_dummy.set_index('item'), on='item', how = "left")
#ratings = ratings.join(items_dummy.set_index('item'), on='item', how = "left")
ratings1 = ratings1.dropna().reset_index(drop="true")
ratings1.head()
#df1 = df[['a','b']]
ratings = ratings1[[ "user", "item", "rating"]]
ratings.head()

,user,item,rating
0,186,302,3
1,22,377,1
2,166,346,1
3,298,474,4
4,115,265,2


In [7]:
algo_uu = uknn.UserUser(20)
algo_ii = iknn.ItemItem(20)
algo_als = als.BiasedMF(50)
algo_funkSVD = funksvd.FunkSVD(50)


In [14]:
items_n = ratings.item.nunique()
items_n


1337

In [15]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    
    # now we run the recommender
    recs = batch.recommend(fittable, users, items_n)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [16]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    all_recs.append(eval('FunkSVD', algo_funkSVD, train, test))
    

In [17]:
test_data = pd.concat(test_data, ignore_index=True)
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1449,5.034580,1,1,ItemItem,Waiting for Guffman (1996),Comedy,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,483,4.830466,1,2,ItemItem,King of the Hill (1993),Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,973,4.805036,1,3,ItemItem,Meet John Doe (1941),Comedy|Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,408,4.781971,1,4,ItemItem,8 Seconds (1994),Drama,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1398,4.778788,1,5,ItemItem,In Love and War (1996),Romance|War,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [18]:
test_data.to_pickle("test_data_Lenskit2")
all_recs_joined.to_pickle("all_recs_joined_Lenskit2")

In [ ]:
#test = all_recs_joined.loc[all_recs_joined["user"]==1]
#test = test.loc[test["Algorithm"] == "ALS"]
#test.reset_index()
#test.iloc[90:100,:]

In [ ]:
#providers = list(all_recs_joined.iloc[:,8:])
#items_n = all_recs_joined.item.nunique()
#proItems_n =  all_recs_joined.loc[ all_recs_joined['Drama'] == 1].item.nunique()
#proItems_n_horror =  all_recs_joined.loc[ all_recs_joined['Horror'] == 1].item.nunique()

In [ ]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results = rla.compute(all_recs_joined, test_data, "Drama", providers, items_n, proItems_n)
#results.head()

In [ ]:
#results.groupby('Algorithm').mean()

In [ ]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results_horror = rla.compute(all_recs_joined, test_data, "Horror", providers, items_n, proItems_n_horror)
#results_horror.groupby('Algorithm').mean()

In [ ]:
#allResults = []
#allResults.append(results.groupby('Algorithm').mean())
#allResults.append(results_horror.groupby('Algorithm').mean())

In [ ]:
#pd.concat(allResults, keys=['Drama', 'Horror'])

In [ ]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [ ]:
testproviders = ["Drama", "Action"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

In [ ]:
output = pd.concat(allResults, keys=testproviders)
output